# Disaster Region Selection Tool
_____________________________________________

### Import libraries

In [2]:
import pandas as pd
import numpy as np
import json
import plotly.graph_objs as go
from ipywidgets import interactive, HBox, VBox

### Load data

In [3]:
# income data
income = pd.read_csv('./Data/complete.csv')

# state occupation category statistics
industries = pd.read_csv('./Data/industries.csv')

# county geojson
counties = pd.read_csv('./Data/counties.csv')
counties.drop([2014,2019,2023,2809],inplace = True)
counties.reset_index(inplace = True,drop = True)
counties['dictionaries'] = [json.loads(counties['dictionaries'][i].replace("'", "\"")) for i in range(counties.shape[0])]

# state centroids
with open('./Data/state_centroids.json') as file:
    state_cent = json.loads(file.read())

In [53]:
def summary_stats():
    tot_income = sum(select[0][0]['cells']['values'][1])
    tot_population = sum(select[0][0]['cells']['values'][2])
    all_cities = [i[0] + i[1:].lower() for i in list(set(select[0][0]['cells']['values'][3]))]
    print('Total income yearly income : ' + str(int(tot_income/1000)) + ' million dollars' + '\n' + 
          'Total population concerned : ' + str(int(tot_population/1000)) + ' 000 people' + '\n' + 
          'Cities concerned : ' + str(all_cities))

_____________________________________________
## Initialize Visualization
#### Initialize on California

In [4]:
mapbox_access_token = 'pk.eyJ1IjoiYmRtaXJhbmRhNTgiLCJhIjoiY2pxbGR6MHExMXJzbTN5bDJmODUxcGVqaiJ9.zatbYfSz-EQ8lTzXkd8LZA'

In [5]:
state_selected = 'CA'

# initialize state centroid 
state_selected_lat = state_cent[state_selected]['lat']
state_selected_lon = state_cent[state_selected]['lon']

# initialize dataframes 
income_state_selected = income[income['state'] == state_selected].sort_values(by='zipcode').reset_index(drop=True)
industries_state_selected = industries[industries['state'] == state_selected]
counties_state_selected = counties[counties['states'] == state_selected]

# initialize lat/lon of zip codes in CA
lats = income[income['state'] == 'CA']['lat']
lons = income[income['state'] == 'CA']['lon']

### Build Choropleth

In [6]:
# build color palette
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6',
          '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3',
          '#808000', '#ffd8b1', '#000075', '#808080']

# build zip code nodes for selection on choropleth
zipcode_nodes = dict(type='scattermapbox',
                lat=lats, 
                lon=lons,  
                mode='markers',
                marker=dict(size=1),
                showlegend=False,
                hoverinfo='none'
                )
# build county map top layer for selected state
layers=[dict(sourcetype = 'geojson',
             source = counties_state_selected['dictionaries'].tolist()[i],  
             below = 'water', 
             type = 'fill',
             color = np.random.choice(colors, replace=True),
             opacity=0.5
            ) for i in range(len(counties_state_selected['dictionaries'].tolist()))]

# build base map and zoom to selected state
layout = dict(title=f'{state_selected} State Zip Code Selection Tool<br>for Disaster Income Loss', 
              font=dict(family='Balto'),
              autosize=False,
              width=800,
              height=800,
              hovermode='closest',
   
              mapbox=dict(accesstoken=mapbox_access_token,
                          layers=layers,
                          bearing=0,
                          center=dict(
                          lat=state_selected_lat, 
                          lon=state_selected_lon), 
                          pitch=0,
                          zoom=5,
                    ) 
              )
# combine zip code nodes, county layer, and map into a figure
fig = dict(data=[zipcode_nodes], layout=layout)

# assign Figure Widget for interactions
fw=go.FigureWidget(fig)
choro = fw.data[0]


# create table that displays estimated income loss by zipcode and updates based on state and zipcode selection
t1 = go.FigureWidget([go.Table(
     header=dict(values=['Affected ZIP Codes', 'Yearly Estimated Income Risk', 'Population', 'City', 'County'],
                 fill = dict(color='#C2D4FF'),
                 align = ['left'] * 5),
     cells=dict(values=[income_state_selected['zipcode'],
                       income_state_selected['tot_income'],
                       income_state_selected['population'],
                       income_state_selected['city'],
                       income_state_selected['county']],
                fill = dict(color='#F5F8FF'),
                align = ['left'] * 5))])

# create table that displays income loss based on industry/occupation and updates based on selection
t2 = go.FigureWidget([go.Table(
     header=dict(values=['Occupation', 'Total Employment', 'Portion', 'Average Hourly Rate', 'Average Annual Salary', 
                         '10th Percentile', '25th Percentile', 'Median', '75th Percentile', '90th Percentile'],
                 fill = dict(color='#a1c3d1'),
                 align = ['left'] * 5),
     cells=dict(values=[industries_state_selected['occupation'],
                        industries_state_selected['tot_employement'],
                        industries_state_selected['perc'],
                        industries_state_selected['h_mean'],
                        industries_state_selected['annual_mean'],
                        industries_state_selected['10'],
                        industries_state_selected['25'],
                        industries_state_selected['med'],
                        industries_state_selected['75'],
                        industries_state_selected['90']],
                fill = dict(color='#EDFAFF'),
                align = ['left'] * 5))])

# initiate dictionary for data collection from user selection
select = {}

# update figure and table based on state selected in dropdown
def update_state(state_list):
    choro = fw.data[0]
    
    with fw.batch_update():
        
        # assign state selected from dropdown
        state_selected = state_dropdown.kwargs['state_list']
        
        # updated dataframes and lists based on state selected
        state_selected_lat = state_cent[state_selected]['lat']
        state_selected_lon = state_cent[state_selected]['lon']
        income_state_selected = income[income['state'] == state_selected].sort_values(by='zipcode').reset_index(drop=True)
        industries_state_selected = industries[industries['state'] == state_selected]
        counties_state_selected = counties[counties['states'] == state_selected]
        
        # initialize lat/lon of zip codes in CA
        lats = income[income['state'] == state_selected]['lat']
        lons = income[income['state'] == state_selected]['lon']
        
        # update zipcode nodes
        choro['lat'] = lats
        choro['lon'] = lons
        
        # update county map top layer
        fw['layout']['mapbox']['layers'] = [dict(sourcetype = 'geojson',
                                                 source = counties_state_selected['dictionaries'].tolist()[i],  
                                                 below = 'water', 
                                                 type = 'fill',   
                                                 color = np.random.choice(colors, replace=True),
                                                 opacity=0.5
                                                ) for i in range(len(counties_state_selected['dictionaries'].tolist()))]
        
        # update base map
        fw['layout'] = dict(title=f'{state_selected} State Zip Code Selection Tool<br>for Disaster Income Loss',  
                            font=dict(family='Balto'),
                            autosize=False,
                            width=800,
                            height=800,
                            hovermode='closest',

                            mapbox=dict(accesstoken=mapbox_access_token,
                                  layers=layers,
                                  bearing=0,
                                  center=dict(
                                  lat=state_selected_lat, 
                                  lon=state_selected_lon), 
                                  pitch=0,
                                  zoom=5,
                                ) 
                            )
        
        # update table figures
        t1.data[0]['cells']['values'] = (income_state_selected['zipcode'].tolist(),
                                         income_state_selected['tot_income'].tolist(),
                                         income_state_selected['population'].tolist(),
                                         income_state_selected['city'].tolist(),
                                         income_state_selected['county'].tolist())
        
        # update occupation table figures
        t2.data[0]['cells']['values'] = (industries_state_selected['occupation'].tolist(),
                                         industries_state_selected['tot_employement'].tolist(),
                                         industries_state_selected['perc'].tolist(),
                                         industries_state_selected['h_mean'].tolist(),
                                         industries_state_selected['annual_mean'].tolist(),
                                         industries_state_selected['10'].tolist(),
                                         industries_state_selected['25'].tolist(),
                                         industries_state_selected['med'].tolist(),
                                         industries_state_selected['75'].tolist(),
                                         industries_state_selected['90'])
        
        # define function for updates based on zipcode selection
        def selection_fn(trace,points,selector):
            t1.data[0].cells.values = [income_state_selected.loc[points.point_inds, col] 
                                       for col in ['zipcode', 'tot_income', 'population', 'city', 'county']]
            select[0]=t1.data
        
        # execute zipcode selection event
        choro.on_selection(selection_fn)

# assign state dropdown interaction
state_dropdown = interactive(update_state, state_list=sorted(state_cent.keys()))

# auto layout in VBox and HBox
VBox((HBox(state_dropdown.children),fw,t1, t2))

VBox(children=(HBox(children=(Dropdown(description='state_list', options=('AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'), value='AK'), Output())), FigureWidget({
    'data': [{'hoverinfo': 'none',
              'lat': array([33.9740268, 33.9490988, 33.9641307, ..., 38.8699434, 39.313089 ,
                            39.313089 ]),
              'lon': array([-118.2495088, -118.2467371, -118.2727831, ..., -120.0089427,
                            -120.2077827, -120.2077827]),
              'marker': {'size': 1},
              'mode': 'markers',
              'showlegend': False,
              'type': 'scattermapbox',
              'uid': '7c3481da-1b4c-11e9-837f-6036ddfde66b'}],
    'layout': {'autosize': False,
               'font': {'family': 'Balto'},
               'height': 800,
               'hovermode': 'closest',
               'mapbox': {'accesstoken': ('pk.eyJ1IjoiYmRtaXJhbmRhNTgiLCJ' ... 'GVqaiJ9.zatbYfSz-EQ8lTzXkd8LZA'),
                          'bearing': 0,
                          'center': {'lat': 37.1841, 'lon': -119.4696},
                          'layers': [{'below': 'water',
                                      'color': '#808000',
                                      'opacity': 0.5,
                                      'source': {'geometry': {'coordinates': [[[[-
                                                                              124.0
                                                                              548, 
                                                                              41.46
                                                                              4], [
                                                                              -123.
                                                                              77, 4
                                                                              1.464
                                                                              ], [-
                                                                              123.7
                                                                              7, 41
                                                                              .3818
                                                                              ], [-
                                                                              123.6
                                                                              605, 
                                                                              41.38
                                                                              18], 
                                                                              [-123
                                                                              .4742
                                                                              , 41.
                                                                              3654]
                                                                              , [-1
                                                                              23.45
                                                                              78, 4
                                                                              1.233
                                                                              9], [
                                                                              -123.
                                                                              4085,
                                                                              41.17
                                                                              9

In [60]:
summary_stats()

In [61]:
industries[industries['state'] == 'AZ'].drop(['perc','state'],axis =1)